# Segmenting and Clustering Neighborhoods in Toronto

In [1]:
!pip install bs4

In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

In [3]:
Canada_M = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(Canada_M, "html")

In [4]:
PostalCode = []
Borough = []
Neighborhood = []

for data in soup.tbody.find_all('tr')[1:]:
    for td in data.find_all('td'):
        tdtxt=td.text.replace('\n','').replace(' /',',').replace(')','')
        if len(tdtxt[3:].split('(')[0]) > 0 and tdtxt[3:].split('(')[0] != "Not assigned":
            PostalCode.append(tdtxt[:3])
            Borough.append(tdtxt[3:].split('(')[0])
            if len(tdtxt[3:].split('(')) == 1 or tdtxt[3:].split('(')[1] == "Not assigned" :
                Neighborhood.append(tdtxt[3:].split('(')[0])
            else:
                Neighborhood.append(tdtxt[3:].split('(')[1])
            

In [6]:
alldata = {'PostalCode': PostalCode,
        'Borough': Borough,
        'Neighborhood': Neighborhood}
df = pd.DataFrame(alldata)
# Drop rows where Borough is 'Not assigned', then reset index
#df = df[df.Borough != 'Not assigned'].reset_index().drop('index', axis = 1)
df = df.groupby(df['PostalCode']).aggregate(sum).reset_index()
df.head(13)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [7]:
df.shape

(97, 3)

In [9]:
!pip install geocoder

In [67]:
import geocoder # import geocoder
def getlatlong(postal_code):
    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
        print('postal_code',postal_code)
        if lat_lng_coords is None:
            continue
        latitude = lat_lng_coords[0]
        longitude = lat_lng_coords[1]
    print(latitude,'and',longitude)
    return latitude, longitude

longlat=pd.read_csv('http://cocl.us/Geospatial_data')
longlat=longlat.set_index('Postal Code')
def getlnglatfromfile(post):
    latitude = longlat.loc[post]['Latitude']
    longitude = longlat.loc[post]['Longitude']
    return latitude, longitude


In [68]:
df[['Latitude', 'Longitude']]=df['PostalCode'].apply(lambda x: pd.Series(getlnglatfromfile(x)))

In [69]:
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [70]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

import folium # plotting library


In [71]:
CLIENT_ID = 'PESQIFIZTNDBTMZPSPLZXDCQVGVQT50IE3K1RP3QCQALSDCV' # your Foursquare ID
CLIENT_SECRET = 'ODUYPM1K0ZG1Z0LZTM4GE5IRA0KSMBARQMVDJWAD0HC04UBW' # your Foursquare Secret
VERSION = '20181018'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PESQIFIZTNDBTMZPSPLZXDCQVGVQT50IE3K1RP3QCQALSDCV
CLIENT_SECRET:ODUYPM1K0ZG1Z0LZTM4GE5IRA0KSMBARQMVDJWAD0HC04UBW


In [81]:
centerlat=df['Latitude'].mean()
centerlong=df['Longitude'].mean()
df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
92,M9N,York,Weston,43.706876,-79.518188
93,M9P,Etobicoke,Westmount,43.696319,-79.532242
94,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
95,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


In [86]:
venues_map = folium.Map(location=[centerlat, centerlong], zoom_start=10) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the center of all postal code regions
folium.CircleMarker(
    [centerlat, centerlong],
    radius=10,
    color='red',
    popup='the Center',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label, Bor in zip(df.Latitude, df.Longitude, df.PostalCode, df.Borough):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label+' '+Bor,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map